In [13]:
from torch.torch_version import TorchVersion
from torch.nn import Conv2d,MaxPool2d,Flatten,Sequential,Dropout
from torch.utils.data import DataLoader,Dataset
import time
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import copy
from scipy.io import loadmat
import math

**just model**

In [14]:
class CNN_2D(nn.Module):
  def __init__(self):
    super(CNN_2D,self).__init__()
    self.FE = nn.Sequential(
        nn.Conv2d(in_channels=200,out_channels=24,kernel_size=(3,3),stride=(1,1),padding=(1,1),bias=False),
        nn.BatchNorm2d(24),
    )
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels=24,out_channels=48,kernel_size=(3,3),stride=(1,1),padding=(1,1)),
        nn.BatchNorm2d(48),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=(2,2),stride=(2,2)),
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels=48,out_channels=96,kernel_size=(3,3),stride=(1,1),padding=(1,1)),
        nn.BatchNorm2d(96),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=(2,2),stride=(2,2)),
        nn.Dropout2d(p=0.5),
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(in_channels=96,out_channels=192,kernel_size=(3,3),stride=(1,1),padding=(1,1)),
        nn.BatchNorm2d(192),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=(2,2),stride=(2,2)),
        nn.Dropout2d(p=0.5),
    )
    self.classifier = nn.Linear(27648,16)
  def forward(self, x):
    print("x.shape:",x.shape)
    x_FE = self.FE(x)
    print("x_FE.shape:",x_FE.shape)
    x_1 = self.layer1(x_FE)
    print("x_1.shape",x_1.shape)
    x_2 = self.layer2(x_1)
    print("x_2.shape",x_2.shape)
    x_3 = self.layer3(x_2)
    print("x_3.shape",x_3.shape)
    x_end = torch.reshape(x_3,(x_3.shape[0],-1))
    print("x_end.shape",x_end.shape)
    x_out = self.classifier(x_end)
    print("x_out.shape",x_out.shape)
    return x_out

In [15]:
# 随机输入，测试网络结构是否通
x = torch.randn(4, 200, 100, 100)
net = CNN_2D()
y = net(x)
print("y.shape",y.shape)

x.shape: torch.Size([4, 200, 100, 100])
x_FE.shape: torch.Size([4, 24, 100, 100])
x_1.shape torch.Size([4, 48, 50, 50])
x_2.shape torch.Size([4, 96, 25, 25])
x_3.shape torch.Size([4, 192, 12, 12])
x_end.shape torch.Size([4, 27648])
x_out.shape torch.Size([4, 16])
y.shape torch.Size([4, 16])


In [16]:
import torch
from torch import nn

net = CNN_2D()

from torchsummary import summary
print(summary(net,input_size=(200, 100, 100)))


x.shape: torch.Size([2, 200, 100, 100])
x_FE.shape: torch.Size([2, 24, 100, 100])
x_1.shape torch.Size([2, 48, 50, 50])
x_2.shape torch.Size([2, 96, 25, 25])
x_3.shape torch.Size([2, 192, 12, 12])
x_end.shape torch.Size([2, 27648])
x_out.shape torch.Size([2, 16])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 24, 100, 100]          43,200
       BatchNorm2d-2         [-1, 24, 100, 100]              48
            Conv2d-3         [-1, 48, 100, 100]          10,416
       BatchNorm2d-4         [-1, 48, 100, 100]              96
              ReLU-5         [-1, 48, 100, 100]               0
         MaxPool2d-6           [-1, 48, 50, 50]               0
            Conv2d-7           [-1, 96, 50, 50]          41,568
       BatchNorm2d-8           [-1, 96, 50, 50]             192
              ReLU-9           [-1, 96, 50, 50]               0
        MaxPool2d-10           